# Votaciones Diputados Argentina Data Discovery

En este Jupyter Notebook, vamos a descubrir metadata sobre el dataset de las votaciones de legisladores argentinos. En este [link](https://github.com/nahuelhds/votaciones-diputados-argentina) se pueden encontrar más detalles sobre esto.

## Data Discovery
Para explorar los datos se utilizará [Pandas](https://pandas.pydata.org/), una librería de Python muy conocida. 

Antes de empezar, algunos imports...

In [96]:
import os
import urllib.request
import pandas as pd
import plotly as py

Y se declaran algunas variables útiles:

In [97]:
import_directory = os.path.join("/tmp", "datasets", "votaciones")

dataset_base_url = "https://raw.githubusercontent.com/nahuelhds/votaciones-diputados-argentina"
dataset_path = "/master/data/csv/"
full_path = dataset_base_url + dataset_path

# You can use as many datasets as you want here. They'll all be cached in your file system.
datasets = [
    "votaciones_votos-2015.csv",
    "votaciones_votos-2016.csv",
    "votaciones_votos-2017.csv",
    "votaciones_votos-2018.csv"
]

# Dataset Metadata

Generamos metadata temporal sobre los datasets. Esta metadata es útil a la hora de descargar y cachear los datasets en el file system.

In [98]:

# { dataset: { url: http://..., file_name: /tmp/datasets/...} }
datasets_metadata = {}

# Populate dataset metadata.
for dataset in datasets:
    dataset_file_name = os.path.join(import_directory, dataset)
    dataset_url = full_path + dataset
    datasets_metadata.update({dataset: {
        'url': dataset_url,
        'file_name': dataset_file_name
    }})

# Descarga de Datasets en carpeta temporal

In [111]:
def download_dataset( dataset_url, file_name):
    print("Downloading " + dataset_url + " into " + file_name)
    urllib.request.urlretrieve(dataset_url, file_name)

In [100]:
def download_datasets():
    print("Downloading Datasets from " + full_path)
    for dataset in datasets:
        dataset_metadata = datasets_metadata[dataset]
        download_dataset(dataset_metadata.get('url'), dataset_metadata.get('file_name'))
    print("Datasets downloaded")


In [101]:
def load_datasets():
    dfs = []
    for dataset in datasets:
        dataset_filename = datasets_metadata[dataset].get('file_name')
        if not dfs:
            # first dataset with the headers
            df = pd.read_csv(dataset_filename)
        else:
            # the rest without headers
            df = pd.read_csv(dataset_filename, index_col=None, header=0)
        dfs.append(df)
    final_df = pd.concat(dfs, axis=0, ignore_index=True)
    return final_df

# Creamos carpeta temporal si no existe y descargamos los datasets

In [102]:
if not os.path.isdir(import_directory):
    print("Creating tmp directory")
    os.makedirs(import_directory)
    download_datasets()

# Simple Visualizacion de los Datasets

In [103]:
votos = load_datasets()
print("##### Votos de ejemplo:#######")
print(votos.head())
print("\n#######Resumen del dataset######")
print(votos.info())

##### Votos de ejemplo:#######
       id  voting_id  legislador_id                         legislador  \
0  458450       1785           1156  ABDALA DE MATARAZZO, Norma Amanda   
1  458451       1785           1337                 ABRAHAM, Alejandro   
2  458452       1785            930                  AGUAD, Oscar Raúl   
3  458453       1785           1247               AGUILAR, Lino Walter   
4  458454       1785           1338             ALEGRE, Gilberto Oscar   

   bloque_id                        bloque  provincia_id            provincia  \
0        119    Frente Cívico por Santiago            22  Santiago del Estero   
1          1  Frente para la Victoria - PJ            13              Mendoza   
2          6          Unión Cívica Radical             3              Córdoba   
3        197            Compromiso Federal            19             San Luis   
4         85              Frente Renovador             1         Buenos Aires   

         voto  
0  AFIRMATIVO  
1  AF

# Visualizacion usando Plotly

Usando `plotly`, se pueden visualizar los votos del bloque o de un legislador:

In [104]:
# set up de plotly
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
init_notebook_mode(connected=True)

In [105]:
# Creamos tmp folder
if not os.path.exists('images'):
    os.mkdir('images')

In [106]:
# De todos los campos solo queremos bloque y voto
subset = votos.filter(["bloque", "voto"])

positivos = subset['voto'] == 'AFIRMATIVO'
negativos = subset['voto'] == 'NEGATIVO'
ausentes = subset['voto'] == 'AUSENTE'

In [107]:
# creamos 3 datasets: afirmativo, negativo y ausente
positivos_por_bloque = subset[positivos].groupby('bloque')
negativos_por_bloque = subset[negativos].groupby('bloque')
ausentes_por_bloque = subset[ausentes].groupby('bloque')

In [108]:
# Realizamos el cuento de votos por bloque y graficamos
positivos_count = positivos_por_bloque.agg('count')

labels = positivos_count.index.values
values = positivos_count.values.flatten()

trace = go.Pie(labels=labels, values=values, title='Votos Positivos')

py.offline.iplot([trace], filename='positivos')

In [109]:
negativos_count = negativos_por_bloque.agg('count')

labels = negativos_count.index.values
values = negativos_count.values.flatten()

trace = go.Pie(labels=labels, values=values, title='Votos Negativos')

py.offline.iplot([trace], filename='negativos')

In [110]:
ausentes_count = ausentes_por_bloque.agg('count')

labels = ausentes_count.index.values
values = ausentes_count.values.flatten()

trace = go.Pie(labels=labels, values=values, title='Votos Ausentes')

py.offline.iplot([trace], filename='ausentes')

# Text Mining
El objetivo es tokenizar nombre de los partidos politicos y extraer semántica.

In [204]:
bloques = votos['bloque'].drop_duplicates()

In [206]:
#print(bloques)

In [207]:
print(bloques.head())

0      Frente Cívico por Santiago
1    Frente para la Victoria - PJ
2            Unión Cívica Radical
3              Compromiso Federal
4                Frente Renovador
Name: bloque, dtype: object


# TextBlob
Text Blob es una librería 

# Install TextBlob in Anaconda
Puede que sea necesario instalar manualmente TextBlob: https://anaconda.org/conda-forge/textblob

In [208]:
from textblob import TextBlob

In [209]:
bloque = bloques.iloc[0]

In [210]:
print(bloque)

Frente Cívico por Santiago


In [163]:
import nltk
nltk.download('punkt')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [154]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/arielsegura/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [157]:
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     /Users/arielsegura/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

# Stopwords
Eliminamos stopwords de los bloques

In [165]:
from nltk.corpus import stopwords 

In [168]:
stop_words = set(stopwords.words('spanish'))

In [172]:
#print(stop_words)

In [147]:
blob = TextBlob(bloque)

In [148]:
print(blob)

Frente Cívico por Santiago


In [211]:
print(bloques.head())

0      Frente Cívico por Santiago
1    Frente para la Victoria - PJ
2            Unión Cívica Radical
3              Compromiso Federal
4                Frente Renovador
Name: bloque, dtype: object


In [245]:
bloques_text_blob = bloques.map(lambda x: TextBlob(str(x)))


In [259]:
print(bloques_text_blob.map(lambda x: {str(x): x.sentiment}))

0                {'Frente Cívico por Santiago': (0.0, 0.0)}
1              {'Frente para la Victoria - PJ': (0.0, 0.0)}
2                      {'Unión Cívica Radical': (0.0, 0.0)}
3                        {'Compromiso Federal': (0.0, 0.0)}
4                          {'Frente Renovador': (0.0, 0.0)}
6                                 {'Unión PRO': (0.0, 0.0)}
9                              {'Proyecto Sur': (0.0, 0.0)}
11                                     {'UNIR': (0.0, 0.0)}
15                       {'Partido Socialista': (0.0, 0.0)}
27                           {'U.De.So. Salta': (0.0, 0.0)}
29              {'Movimiento Popular Fueguino': (0.6, 0.9)}
31                {'PTS - Frente de Izquierda': (0.0, 0.0)}
32        {'Fte. Cívico y Social de Catamarca': (0.03333...
34                                       {'Fe': (0.0, 0.0)}
47            {'Coalición Cívica - ARI - UNEN': (0.0, 0.0)}
48                              {'SUMA + UNEN': (0.0, 0.0)}
53                        {'Unión por Có

In [261]:
print(bloques_text_blob.map(lambda x: x.words))

0                        [Frente, Cívico, por, Santiago]
1                       [Frente, para, la, Victoria, PJ]
2                               [Unión, Cívica, Radical]
3                                  [Compromiso, Federal]
4                                    [Frente, Renovador]
6                                           [Unión, PRO]
9                                        [Proyecto, Sur]
11                                                [UNIR]
15                                 [Partido, Socialista]
27                                      [U.De.So, Salta]
29                       [Movimiento, Popular, Fueguino]
31                          [PTS, Frente, de, Izquierda]
32               [Fte, Cívico, y, Social, de, Catamarca]
34                                                  [Fe]
47                        [Coalición, Cívica, ARI, UNEN]
48                                          [SUMA, UNEN]
53                                 [Unión, por, Córdoba]
61                       [Movim

In [262]:
bloques_sin_stop_words = bloques_text_blob.map(lambda blob: 
                                                    [word for word in blob.words if word not in stop_words])


In [263]:
print(bloques_sin_stop_words)

0                         [Frente, Cívico, Santiago]
1                             [Frente, Victoria, PJ]
2                           [Unión, Cívica, Radical]
3                              [Compromiso, Federal]
4                                [Frente, Renovador]
6                                       [Unión, PRO]
9                                    [Proyecto, Sur]
11                                            [UNIR]
15                             [Partido, Socialista]
27                                  [U.De.So, Salta]
29                   [Movimiento, Popular, Fueguino]
31                          [PTS, Frente, Izquierda]
32                  [Fte, Cívico, Social, Catamarca]
34                                              [Fe]
47                    [Coalición, Cívica, ARI, UNEN]
48                                      [SUMA, UNEN]
53                                  [Unión, Córdoba]
61                   [Movimiento, Popular, Neuquino]
66                              [Unión, Entre,

# NLTK Tags

- CC coordinating conjunction
- CD cardinal digit
- DT determiner
- EX existential there (like: “there is” … think of it like “there exists”)
- FW foreign word
- IN preposition/subordinating conjunction
- JJ adjective ‘big’
- JJR adjective, comparative ‘bigger’
- JJS adjective, superlative ‘biggest’
- LS list marker 1)
- MD modal could, will
- NN noun, singular ‘desk’
- NNS noun plural ‘desks’
- NNP proper noun, singular ‘Harrison’
- NNPS proper noun, plural ‘Americans’
- PDT predeterminer ‘all the kids’
- POS possessive ending parent‘s
- PRP personal pronoun I, he, she
- PRP possessive pronoun my, his, hers
- RB adverb very, silently,
- RBR adverb, comparative better
- RBS adverb, superlative best
- RP particle give up
- TO to go ‘to‘ the store.
- UH interjection errrrrrrrm
- VB verb, base form take
- VBD verb, past tense took
- VBG verb, gerund/present participle taking
- VBN verb, past participle taken
- VBP verb, sing. present, non-3d take
- VBZ verb, 3rd person sing. present takes
- WDT wh-determiner which
- WP wh-pronoun who, what
- WP possessive wh-pronoun whose
- WRB wh-abverb where, when

Source: https://www.geeksforgeeks.org/part-speech-tagging-stop-words-using-nltk-python/